In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
% cd '/content/gdrive/MyDrive/Char_RNN'

/content/gdrive/MyDrive/Char_RNN


In [4]:
with open('text.txt', 'r') as f:
    text = [line for line in f]

print(text[10:15])

['the delegates from the Orient, have told you that these men from far-off \n', 'nations may well claim the honour of bearing to different lands the idea of \n', 'toleration. I am proud to belong to a religion which has taught the world both \n', 'tolerance and universal acceptance. We believe not only in universal toleration, \n', 'but we accept all religions as true. I am proud to belong to a nation which has \n']


In [5]:
text[30:32]

['doctrine preached in the Gita: “Whosoever comes to Me, through whatsoever \n',
 'form, I reach him; all men are struggling through paths which in the end lead to \n']

In [6]:
prev = -1 #index of last group of '\n'
prev_word_idx = 1 #index of the last item that was not '\n' 
prev_n = -1 
continuous= 0 #counting the consecutive '\n'
list_idx_to_remove = []
for i in range(2, len(text)):
  if text[i]=='\n':
    continuous+= 1
  else:
    prev_word_idx = i
    continuous = 0
  if continuous>1:
    if (prev> -1) and (i - prev)==5:
      if len(text[i-3])<40 and text[i-3]!='\n':
        list_idx_to_remove.append(i-3)
        #remove next sentence if only the first is removed
        if len(text[i-2])<40 and text[i-2]!='\n':
          list_idx_to_remove.append(i-2)
    if(prev>-1) and (i-prev)==4:
      if len(text[i-2])<40 and text[i-2]!='\n':
        list_idx_to_remove.append(i-2)
    if continuous>2:
      list_idx_to_remove.append(prev_word_idx)
    prev = i-continuous + 1

print(len(list_idx_to_remove), list_idx_to_remove[:5])

2782 [45, 46, 92, 540, 541]


In [7]:
list_idx_to_remove = list(set(list_idx_to_remove))
list_idx_to_remove.sort()
print(len(list_idx_to_remove))
for idx_to_remove in list_idx_to_remove[11:13]:
  print(text[idx_to_remove -3:idx_to_remove+3])
  print(text[idx_to_remove])

2362
['renunciation. \n', '\n', '\n', 'Chapter IV. What Is Duty? \n', '\n', '\n']
Chapter IV. What Is Duty? 

['from each other, they at last lead to the same goal of human perfection. \n', '\n', '\n', 'Chapter VII. Freedom \n', '\n', '\n']
Chapter VII. Freedom 



In [8]:
cleaned_text = []
continuous = 0 #the number of consecutive '\n'

for i in range(len(text)):
  line = text[i]
  if line=='\n':
    continuous +=1
  else:
    continuous = 0
  
  if i in list_idx_to_remove:
    continue
  
  if continuous>3:
    continue
  cleaned_text.append(line)

print(len(cleaned_text), len(text))

146561 149291


In [9]:
cleaned_text = "".join(cleaned_text)

In [10]:
import numpy as np
import pandas as pd
import torch
import torchtext
from collections import defaultdict
import random

In [11]:
cleaned_text = cleaned_text.lower()

In [12]:
chars = list(set(cleaned_text))

int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in cleaned_text])
encoded[:50]

array([53, 40, 32, 40,  0, 28, 34, 40, 25, 55, 81, 61, 25, 48, 34, 10,  0,
       27, 28, 34, 40, 25, 10, 50, 25, 55, 78, 28, 34, 32, 74, 55, 62, 25,
       53, 53, 32,  0, 25, 50, 32,  3,  3, 40, 25, 78, 71, 25, 27, 28])

In [13]:
def one_hot_encode(arr, n_labels):

  one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
  one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
  one_hot = one_hot.reshape((*arr.shape, n_labels))

  return one_hot

In [14]:
test_seq = np.array([[2, 6, 5]])
one_hot = one_hot_encode(test_seq, 8)
print(one_hot, one_hot.shape)



[[[0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]]] (1, 3, 8)


In [15]:
def get_batches(arr, batch_size, seq_length):
    '''returns batches of size batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded in a sequence
    '''

    total_batch_size = batch_size*seq_length
    num_batches = len(arr)//total_batch_size

    batches = []
    for i in range(0, len(arr), total_batch_size):
      if i+total_batch_size>len(arr):
        break
      chars_of_batch = arr[i:i+total_batch_size]

      x = []
      #x = [chars_of_batch[j*seq_length:j*seq_length] for j in range(batch_size)]
      for j in range(batch_size):
        one_seq = chars_of_batch[j*seq_length:(j+1)*seq_length]
        x.append(one_seq)
      x = np.array(x)

      #print(x, x.shape)
      y = np.zeros_like(x)
      try:
        y[:-1, -1] = x[1:, 0]
      except:
        print(x.shape, y.shape, i, len(arr), total_batch_size)
        y[:-1, -1] = x[1:, 0]
      try:
        y[:, :-1], y[-1, -1] = x[:, 1:], arr[i*seq_length]
      except IndexError:
        y[:, :-1], y[-1, -1] = x[:, 1:], arr[i+total_batch_size-1]
      
      
      #print(y, y.shape)

      yield x, y
      
      
        


In [16]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)
print(x.shape, y.shape)

(8, 50) (8, 50)


In [17]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU.')

Training on GPU!


In [18]:
import torch.nn as nn

In [19]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2, dropout=0.2, lr=0.0001):
        super().__init__()
        self.drop_prob = dropout
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=dropout, batch_first=True)
        
        self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        
        r_output, hidden = self.lstm(x, hidden)
        
        out = self.dropout(r_output)
        
        
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        
        # out.shape = [batch_size*seq_length, len(self.chars)]
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [20]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    prime = prime.lower()
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [21]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        #print(inputs.shape, "input.shape")
        #print(h[0].shape, h[1].shape, "hidd.shape")
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [37]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.2, print_every=300):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    global min_val_loss
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            h = tuple([each.data for each in h])

            net.zero_grad()
            #print(inputs.shape, "inputs")
            output, h = net(inputs, h)
            #print(output.shape, "outputs")
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            
            if counter % print_every == 0:
                sample_written = sample(net, 200, prime='The people', top_k=3)
                try:
                  with open('Example2.txt', 'w') as writefile:
                    writefile.write(sample_written+"\n")
                except:
                  print(sample_written[:75])
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                if np.mean(val_losses)<min_val_loss and round(random.random())==1:
                  min_val_loss = np.mean(val_losses)
                  model_name = 'rnn_22_epoch.net'
                  checkpoint = {'n_hidden': net.n_hidden,
                                'n_layers': net.n_layers,
                                'state_dict': net.state_dict(),
                                'tokens': net.chars}

                  with open(model_name, 'wb') as f:
                    torch.save(checkpoint, f)
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [130]:
n_hidden=512
n_layers=3

net = CharRNN(chars, n_hidden, n_layers)
print(net)
total_params = sum(
	param.numel() for param in net.parameters()
)
print(total_params)

CharRNN(
  (lstm): LSTM(82, 512, num_layers=3, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=512, out_features=82, bias=True)
)
5465170


In [131]:
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [132]:
from torch.optim.lr_scheduler import ExponentialLR 

In [133]:
batch_size = 64
seq_length = 300
n_epochs = 25 # start small if testing initial behavior
# train the model
min_val_loss = 1e7
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=300)

Epoch: 1/25... Step: 300... Loss: 2.4945... Val Loss: 2.4410
Epoch: 2/25... Step: 600... Loss: 1.8750... Val Loss: 1.8997
Epoch: 3/25... Step: 900... Loss: 1.6730... Val Loss: 1.6434
Epoch: 4/25... Step: 1200... Loss: 1.4784... Val Loss: 1.5463
Epoch: 5/25... Step: 1500... Loss: 1.3799... Val Loss: 1.4735
Epoch: 6/25... Step: 1800... Loss: 1.2502... Val Loss: 1.4329
Epoch: 7/25... Step: 2100... Loss: 1.2821... Val Loss: 1.3875
Epoch: 8/25... Step: 2400... Loss: 1.2669... Val Loss: 1.3341
Epoch: 8/25... Step: 2700... Loss: 1.2651... Val Loss: 1.3220
Epoch: 9/25... Step: 3000... Loss: 1.4307... Val Loss: 1.3060
Epoch: 10/25... Step: 3300... Loss: 1.1999... Val Loss: 1.2801
Epoch: 11/25... Step: 3600... Loss: 1.2876... Val Loss: 1.2830
Epoch: 12/25... Step: 3900... Loss: 1.1425... Val Loss: 1.2764
Epoch: 13/25... Step: 4200... Loss: 1.2640... Val Loss: 1.2815
Epoch: 14/25... Step: 4500... Loss: 1.0760... Val Loss: 1.2610
Epoch: 15/25... Step: 4800... Loss: 1.0744... Val Loss: 1.2435
Epoch

In [134]:
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [135]:
print(sample(net, 150, prime='The world', top_k=3))

the worldly spirituality is to be 
able. the soul which is not to be coloured, as the same way as it were and 
they will be superior to the senses as it is. if 


In [136]:
import torch.nn.functional as F

In [137]:
n_hidden=256
n_layers=6

net = CharRNN(chars, n_hidden, n_layers)
print(net)
total_params = sum(
	param.numel() for param in net.parameters()
)
print(total_params)

CharRNN(
  (lstm): LSTM(82, 256, num_layers=6, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=82, bias=True)
)
3000914


In [138]:
batch_size = 128
seq_length = 250
n_epochs = 30 # start smaller if you are just testing initial behavior
# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.002, print_every=300)

Epoch: 2/30... Step: 300... Loss: 2.9625... Val Loss: 3.0162
Epoch: 3/30... Step: 600... Loss: 2.9813... Val Loss: 3.0118
Epoch: 5/30... Step: 900... Loss: 2.9417... Val Loss: 3.0208
Epoch: 6/30... Step: 1200... Loss: 3.0088... Val Loss: 3.0064
Epoch: 8/30... Step: 1500... Loss: 2.9514... Val Loss: 3.0132
Epoch: 9/30... Step: 1800... Loss: 3.0102... Val Loss: 3.0092
Epoch: 11/30... Step: 2100... Loss: 2.9299... Val Loss: 3.0158
Epoch: 12/30... Step: 2400... Loss: 2.9552... Val Loss: 3.0090
Epoch: 14/30... Step: 2700... Loss: 2.9294... Val Loss: 3.0095
Epoch: 15/30... Step: 3000... Loss: 2.9756... Val Loss: 3.0089
Epoch: 17/30... Step: 3300... Loss: 2.9372... Val Loss: 3.0132
Epoch: 18/30... Step: 3600... Loss: 2.9597... Val Loss: 3.0135
Epoch: 20/30... Step: 3900... Loss: 2.9461... Val Loss: 3.0080
Epoch: 21/30... Step: 4200... Loss: 2.9392... Val Loss: 3.0139
Epoch: 22/30... Step: 4500... Loss: 2.9700... Val Loss: 3.0087
Epoch: 24/30... Step: 4800... Loss: 2.9410... Val Loss: 3.0177
E

In [23]:
n_hidden=1024
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)
total_params = sum(
	param.numel() for param in net.parameters()
)
print(total_params)

CharRNN(
  (lstm): LSTM(82, 1024, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=1024, out_features=82, bias=True)
)
13019218


In [25]:
import torch.nn.functional as F

In [27]:
batch_size = 128
seq_length = 400
n_epochs = 45 # start smaller if just testing initial behavior
# train the model
min_val_loss = 1.2435
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.0005, print_every=300)

Epoch: 3/45... Step: 300... Loss: 1.4345... Val Loss: 1.5807
Epoch: 5/45... Step: 600... Loss: 1.3657... Val Loss: 1.4212
Epoch: 8/45... Step: 900... Loss: 1.2018... Val Loss: 1.3517
Epoch: 10/45... Step: 1200... Loss: 1.1286... Val Loss: 1.3066
Epoch: 12/45... Step: 1500... Loss: 1.2987... Val Loss: 1.2708
Epoch: 15/45... Step: 1800... Loss: 1.1121... Val Loss: 1.2590
Epoch: 17/45... Step: 2100... Loss: 1.0556... Val Loss: 1.2429
Epoch: 19/45... Step: 2400... Loss: 1.1024... Val Loss: 1.2213
Epoch: 22/45... Step: 2700... Loss: 1.0706... Val Loss: 1.2261
Epoch: 24/45... Step: 3000... Loss: 1.0960... Val Loss: 1.2180
Epoch: 26/45... Step: 3300... Loss: 1.1464... Val Loss: 1.2043
Epoch: 29/45... Step: 3600... Loss: 1.0042... Val Loss: 1.2271
Epoch: 31/45... Step: 3900... Loss: 1.0651... Val Loss: 1.2174
Epoch: 33/45... Step: 4200... Loss: 1.0310... Val Loss: 1.2088
Epoch: 36/45... Step: 4500... Loss: 0.9233... Val Loss: 1.2389
Epoch: 38/45... Step: 4800... Loss: 0.8975... Val Loss: 1.236

In [33]:
print(sample(net, 150, prime='India is a', top_k=3))

india is a chance of 
practice. the superconscious state of ten times, in all the bad are the 
prophets of the country and the palace of the wave and the protect


In [34]:
with open('input.txt', 'r') as f:
    text = [line for line in f]

print(text[10:15])

['Resolved. resolved.\n', '\n', 'First Citizen:\n', 'First, you know Caius Marcius is chief enemy to the people.\n', '\n']


In [35]:
cleaned_text = "".join(text)

In [38]:
#cleaned_text = cleaned_text.lower()
chars = list(set(cleaned_text))

int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in cleaned_text])
encoded[:50]

array([48, 26, 33, 59,  1,  2, 13, 26,  1, 26, 50, 19, 64,  5, 60, 35, 19,
       55, 24, 33, 19,  2, 17, 19,  2, 20, 33, 24, 49, 19, 19,  9,  2, 63,
       64, 34,  2, 55, 52, 33,  1, 15, 19, 33, 10,  2, 15, 19, 63, 33])

In [41]:
n_hidden=512
n_layers=3

net = CharRNN(chars, n_hidden, n_layers)
print(net)
total_params = sum(
	param.numel() for param in net.parameters()
)
print(total_params)

CharRNN(
  (lstm): LSTM(65, 512, num_layers=3, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=512, out_features=65, bias=True)
)
5421633


In [42]:
batch_size = 64
seq_length = 300
n_epochs = 50 # start small if testing initial behavior
# train the model
min_val_loss = 1e7
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=100)

Epoch: 3/50... Step: 100... Loss: 3.1664... Val Loss: 3.1697
Epoch: 5/50... Step: 200... Loss: 2.7007... Val Loss: 2.6480
Epoch: 7/50... Step: 300... Loss: 2.2959... Val Loss: 2.2797
Epoch: 9/50... Step: 400... Loss: 2.0667... Val Loss: 2.0839
Epoch: 11/50... Step: 500... Loss: 1.9512... Val Loss: 1.9445
Epoch: 14/50... Step: 600... Loss: 1.8157... Val Loss: 1.8405
Epoch: 16/50... Step: 700... Loss: 1.6664... Val Loss: 1.7688
Epoch: 18/50... Step: 800... Loss: 1.6103... Val Loss: 1.7302
Epoch: 20/50... Step: 900... Loss: 1.6141... Val Loss: 1.6811
Epoch: 22/50... Step: 1000... Loss: 1.4051... Val Loss: 1.6434
Epoch: 24/50... Step: 1100... Loss: 1.5521... Val Loss: 1.6124
Epoch: 27/50... Step: 1200... Loss: 1.4112... Val Loss: 1.5905
Epoch: 29/50... Step: 1300... Loss: 1.3120... Val Loss: 1.5754
Epoch: 31/50... Step: 1400... Loss: 1.3400... Val Loss: 1.5751
Epoch: 33/50... Step: 1500... Loss: 1.3570... Val Loss: 1.5705
Epoch: 35/50... Step: 1600... Loss: 1.2466... Val Loss: 1.5678
Epoch

In [65]:
print(sample(net, 150, prime='she is the east', top_k=3))

she is the east,
And with the sea, that will not stay to hear
Where they did be the father that they say a man
We have seen the subjects of the caunter's son.

RICHAR


In [64]:
print(sample(net, 550, prime="""A thing slipp'd idly from me.
Our poesy is as a gum, which oozes
From whence 'tis nourish'd: the fire i' the flint
Shows not till it be struck; our gentle flame""", top_k=3)) #The above string is from an unpublished shapespeare play so is new to the model


a thing slipp'd idly from me.
our poesy is as a gum, which oozes
from whence 'tis nourish'd: the fire i' the flint
shows not till it be struck; our gentle flame
As the most sweet way is a man of that woe
To me to the courage. I am so defence where I stand for the compouse,
And therefore have the world was most abroad;
For what is my brother's face, and they she'll see
What you are the succession of the world with thee
The senaties of his father, to the sea,
And there and there the straight of me to straight
The palace that we with the chambing and
the shepherd's, an all this is the common thousand things
As that I seek to speak. Yet! what, we have some common
Who is that words that thou hast shall not 


The lines that follow the above words are:

*Shows not till it be struck; our gentle flame
Provokes itself and like the current flies
Each bound it chafes. What have you there?*

Our model accurately predicts a complete line. This model has not been trained for long and though it has a significant cross entropy loss, it still writes english pretty correctly especially considering the amount of punctuation Shakespeare used and the incomplete words too!